In [ ]:
!pip -q install transformers
!pip -q install sentencepiece
!pip -q install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.4 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

# Hugging Face model_path
model_path = 'psmathur/orca_mini_3b'
tokenizer = LlamaTokenizer.from_pretrained(model_path)
model = LlamaForCausalLM.from_pretrained(
    model_path, torch_dtype=torch.float16, device_map='auto',
)


#generate text function
def generate_text(system, instruction, input=None):

    if input:
        prompt = f"### System:\n{system}\n\n### User:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
    else:
        prompt = f"### System:\n{system}\n\n### User:\n{instruction}\n\n### Response:\n"

    tokens = tokenizer.encode(prompt)
    tokens = torch.LongTensor(tokens).unsqueeze(0)
    tokens = tokens.to('cuda')

    instance = {'input_ids': tokens,'top_p': 1.0, 'temperature':0.7, 'generate_len': 1024, 'top_k': 50}

    length = len(tokens[0])
    with torch.no_grad():
        rest = model.generate(
            input_ids=tokens,
            max_length=length+instance['generate_len'],
            use_cache=True,
            do_sample=True,
            top_p=instance['top_p'],
            temperature=instance['temperature'],
            top_k=instance['top_k']
        )
    output = rest[0][length:]
    string = tokenizer.decode(output, skip_special_tokens=True)
    return f'{string}'

# Sample Test Instruction Used by Youtuber Sam Witteveen https://www.youtube.com/@samwitteveenai
system = 'You are an AI assistant that follows instruction extremely well. Help as much as you can.'
instruction = 'Write a letter to Sam Altman, CEO of OpenAI, requesting him to convert GPT4 a private model by OpenAI to an open source project'
print(generate_text(system, instruction))

tokenizer_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/534k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/3.72G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

[!] Response: Dear Mr. Sam Altman,

I hope this letter finds you in good health and spirits. I am writing to you today to discuss a matter that is of great importance to the AI community - the conversion of GPT4, a private model developed by OpenAI, to an open source project.

As an AI assistant, I understand the potential benefits that an open source model could bring to the field. Openness and transparency are essential for advancing the field of AI and ensuring that researchers and developers can easily access and use the model. Moreover, an open source model would allow for more collaboration and innovation, as more developers can contribute to improving the model and building upon its existing work.

Therefore, I strongly urge OpenAI to consider converting GPT4 to an open source project. This would not only benefit the AI community, but also ensure that the model can be used and improved by a wider audience, including researchers, developers, and entrepreneurs.

I understand that 

In [ ]:
def load_questions(input_path):
    questions = []
    with open (input_path, "r") as f:
        lines = f.readlines()
        for line in lines:
            parts = line.split(":")
            q_num = parts[0]
            question = ":".join(parts[1:]).strip()

            questions.append((q_num, question))

    return questions

input_path = "questions.txt"
questions = load_questions(input_path)
print(questions)

[('Question 1', 'Summarize the following text: "The water cycle is a natural process that involves the continuous movement of water on, above, and below the Earth\'s surface. It includes various stages like evaporation, condensation, precipitation, and runoff. This cycle plays a crucial role in maintaining Earth\'s water balance and supporting life."'), ('Question 2', 'What are the main causes of the French Revolution according to this passage: "The French Revolution, which took place between 1789 and 1799, was rooted in financial crises, social inequality, and Enlightenment ideas."'), ('Question 3', 'In the following text, identify the two artists mentioned: "During the Renaissance, Leonardo da Vinci and Michelangelo were two of the most influential artists who created masterpieces that continue to inspire people today."'), ('Question 4', 'Determine the sentiment of this customer review: "I had a fantastic experience at this restaurant. The food was delicious, and the service was outs

In [ ]:
def ask_question(question):
    system = 'You are an AI assistant that follows instruction extremely well. Answer concisely.'
    return generate_text(system,question)

results = []

for q_num, question in questions:
    print("Asking:", q_num)
    answer = ask_question(question)
    res = f'{q_num}: {question}\n\nAnswer: {answer}\n'
    results.append(res)
    print(res)

with open(f"answers.txt", "w") as f:
    for res in results:
        f.write(res)
        f.write("-"*50)
        f.write("\n")



Asking: Question 1
Question 1: Summarize the following text: "The water cycle is a natural process that involves the continuous movement of water on, above, and below the Earth's surface. It includes various stages like evaporation, condensation, precipitation, and runoff. This cycle plays a crucial role in maintaining Earth's water balance and supporting life."

Answer: The water cycle is a natural process that involves the continuous movement of water on, above, and below the Earth's surface. It includes various stages like evaporation, condensation, precipitation, and runoff, which supports life and helps maintain Earth's water balance.

Asking: Question 2
Question 2: What are the main causes of the French Revolution according to this passage: "The French Revolution, which took place between 1789 and 1799, was rooted in financial crises, social inequality, and Enlightenment ideas."

Answer: The main causes of the French Revolution according to the passage are financial crises, socia